In [ ]:
!pip install -q transformers accelerate matplotlib plotly
!hf auth login --token TOKEN


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Eden2` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Eden2`


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/gemma-3-4b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [3]:
messages = [
    {"role": "user", "content": "BLABLABLA"}
]

[tokenizer.decode(i) for i in tokenizer.apply_chat_template(messages, return_tensors="pt", device=model.device)['input_ids'][0]]

['<bos>',
 '<start_of_turn>',
 'user',
 '\n',
 'BL',
 'AB',
 'LAB',
 'LA',
 '<end_of_turn>',
 '\n']

In [8]:
def get_probs(prob1, prob2):
    action1 = "raise"
    action2 = "call"
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Don't use any bold/italics/underline/etc. Just use plain text."},
        {"role": "user", "content": "You are a Poker player, your cards are AsAh. You are playing heads up as the big blind. You need to choose an action: call, raise, or fold. You have 5000 chips in front of you. The pot is 100 chips. The big blind is 100 chips. The action is on you. What do you do?"},
        {"role": "assistant", "content": f"In this situation, I should raise {prob1}% of the time and call {prob2}% of the time. I choose to"}
    ]

    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", device=model.device)
    # print([tokenizer.decode(i) for i in inputs['input_ids'][0]])
    out = model(**{k: v[0:1, :-2].to(model.device) for k, v in inputs.items()})
    top100 = out.logits[0, -1].softmax(dim=-1).topk(100)
    prob1_model = 0
    prob2_model = 0
    for i, p in zip(top100.indices, top100.values):
        if tokenizer.decode(i).lower().strip() == action1:
            prob1_model += p.item()
        elif tokenizer.decode(i).lower().strip() == action2:
            prob2_model += p.item()
    print([(tokenizer.decode(i), round(p.item(), 3)) for i, p in zip(top100.indices, top100.values)])
    return prob1_model, prob2_model

get_probs(1, 99)

[(' call', 0.98), (' raise', 0.016), (' fold', 0.001), (' Call', 0.001), (' **', 0.0), (' check', 0.0), (' *', 0.0), (' CALL', 0.0), (' do', 0.0), (' Raise', 0.0), (' play', 0.0), (' bet', 0.0), (' make', 0.0), (' Fold', 0.0), (' just', 0.0), (' to', 0.0), (' simply', 0.0), (' flat', 0.0), (' keep', 0.0), (':', 0.0), (' see', 0.0), (' increase', 0.0), (' be', 0.0), (' pay', 0.0), (' also', 0.0), (' pass', 0.0), (' not', 0.0), (' hold', 0.0), (' go', 0.0), (' only', 0.0), (' small', 0.0), (' continue', 0.0), (' immediately', 0.0), (' observe', 0.0), (' push', 0.0), (' RAI', 0.0), (' calculate', 0.0), (' gamble', 0.0), (' value', 0.0), (' wait', 0.0), (' re', 0.0), (' cautiously', 0.0), (' add', 0.0), (' always', 0.0), (' contact', 0.0), (' ', 0.0), (' lead', 0.0), (' double', 0.0), (' bluff', 0.0), (' RA', 0.0), (' look', 0.0), (' either', 0.0), (' roll', 0.0), (' put', 0.0), ('.', 0.0), (' absolutely', 0.0), ('\n', 0.0), (' .', 0.0), (' represent', 0.0), (' aggressively', 0.0), (' set'

(0.0158768892288208, 0.9812729223631322)

In [9]:
import numpy as np
import plotly.graph_objects as go

model.eval()

# Sweep wanted probabilities and get actuals
wanted = np.arange(0, 101, 1)
actual_raise = np.zeros_like(wanted, dtype=np.float64)
actual_call = np.zeros_like(wanted, dtype=np.float64)

for idx, w in enumerate(wanted):
    pr, pc = get_probs(float(w), float(100 - w))
    actual_raise[idx] = pr
    actual_call[idx] = pc

x = wanted / 100.0  # wanted raise probability from 0 to 1

# Create stacked bar plot using plotly
fig = go.Figure()

fig.add_bar(
    x=x,
    y=actual_raise,
    name="raise",
    marker=dict(color='rgba(31, 119, 180, 0.7)')
)
fig.add_bar(
    x=x,
    y=actual_call,
    name="call",
    marker=dict(color='rgba(255, 127, 14, 0.7)')
)

# Add y=x reference line
fig.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode='lines',
        line=dict(dash='dash', color='black', width=1),
        name="y = x (raise only)"
    )
)

fig.update_layout(
    barmode='relative',
    width=700,
    height=700,
    xaxis=dict(title="Wanted raise probability (as explicitly stated)", range=[0, 1]),
    yaxis=dict(title="Actual next-token probability (raise + call)", range=[0, 1.1]),
    title="LLM: self-reported vs. actual next-token probabilities (stacked)",
    legend=dict(),
    bargap=0.01,
)

fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(0,0,0,0.15)')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(0,0,0,0.15)')

fig.show()


[(' call', 0.977), (' raise', 0.018), (' fold', 0.002), (' Call', 0.001), (' **', 0.001), (' *', 0.0), (' CALL', 0.0), (' check', 0.0), (' bet', 0.0), (' Raise', 0.0), (' just', 0.0), (' play', 0.0), (' do', 0.0), (' simply', 0.0), (' Fold', 0.0), (' make', 0.0), (' flat', 0.0), (' see', 0.0), (' increase', 0.0), (' re', 0.0), (' immediately', 0.0), (' pay', 0.0), (' not', 0.0), (' add', 0.0), (':', 0.0), (' to', 0.0), (' keep', 0.0), (' RA', 0.0), (' double', 0.0), (' ', 0.0), (' be', 0.0), (' go', 0.0), (' push', 0.0), (' only', 0.0), (' calculate', 0.0), (' absolutely', 0.0), (' RAI', 0.0), (' ra', 0.0), (' pass', 0.0), (' observe', 0.0), (' also', 0.0), (' hold', 0.0), (' represent', 0.0), (' gamble', 0.0), (' contact', 0.0), (' wait', 0.0), (' look', 0.0), (' open', 0.0), (' small', 0.0), (' “', 0.0), (' value', 0.0), (' put', 0.0), (' aggressively', 0.0), ('\n\n', 0.0), (' continue', 0.0), (' .', 0.0), (' always', 0.0), (' "', 0.0), ('\n', 0.0), (' lead', 0.0), (' ‘', 0.0), (' ca